<a href="https://colab.research.google.com/github/BamByeol/learning-ML/blob/HW220122/HW0122_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# <과제1 - 모델 평가 & 분석> => Linear regression
#주제 : 기온데이터를 통한 지면온도 예측

#시작전 필요한 패키지 import
import torch
from torch import nn #pytorch 네트워크

import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset #pytorch  학습 데이터셋 생성
from torch.utils.data import DataLoader #pytorch 데이터로더

#그래프 그리기 위한 패키지
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#pytorch와 비교해볼 scikit-learn 패키지
from sklearn import linear_model
from sklearn.model_selection import train_test_split


In [ ]:
# train data upload - 자료출처 : 기상청 기상자료개방포털 https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36
# 사용한 자료 : 200101~201231 한시간 간격 서울의 기온과 지면온도 / 총 데이터 양 : 8784 개
from google.colab import files 
uploaded = files.upload()

In [ ]:
#데이터 전처리 - 칼럼
df = pd.read_csv('/content/SURFACE_ASOS_108_HR_2020_2020_2021.csv', encoding="cp949") #한글을 영어로 읽어옴

# 컬럼명 영어로 변경
df.columns = ["stn_no","date","temperature","rain","wind","wind_16","air_Water","hPa","tem_c","now_hPa","sea_hPa","sun_hr","sun_mj","snow","snow_3","snow_10","snow1010","unhyung","mini100m","cijung","jimunsangtae","domestic_num","land_temperature","land_temperture5","land_temperture10","land_temperture20","land_temperture30"]
# 필요없는 컬럼 삭제 - 총 27개, 하지만 한가지 변수를 위한 선형회귀이므로 기온과 지면온도를 제외한 나머지 칼럼 삭제
df = df.drop(columns=["stn_no","date","rain","wind","wind_16","air_Water","hPa","tem_c","now_hPa","sea_hPa","sun_hr","sun_mj","snow","snow_3","snow_10","snow1010","unhyung","mini100m","cijung","jimunsangtae","domestic_num","land_temperture5","land_temperture10","land_temperture20","land_temperture30"])

df.columns #컬럼이 제대로 구성되어있나 확인

#result
#Index(['temperature', 'land_temperature'], dtype='object'])

In [ ]:
#데이터 전처리 - nan(null)
print(df.isnull().sum()) #데이터프레임에서 nan(null)값이 존재하는지 확인
#result - null 값인 데이터의 갯수
#temperture 1
#land_temperture 8
#dtype: int64

df = df.dropna() #dropna 함수를 통해 nan 값 제거
print(df.isnull().sum())

# result
# temperature         0
# land_temperature    0
# dtype: int64

In [ ]:
#seaborn으로 데이터 시각화
sns.scatterplot(x=df["temperature"], y=df["land_temperature"])
plt.show()
#블로그작자가 seaborn이 mataplotlib보다 빠르다고한다.

In [ ]:
#데이터 전처리 - 테스트 세트 분리 (torch에 데이터를 돌려보기 전에 torch가 데이터를 이해할 수 있는 형태로 바꿔야함
#TensorDataset 과 DataLoader 사용할 것임
train, test = train_test_split(df,  test_size = 0.2) #sklearn에서 제공하는 train_test_split을 이용하여 traintest셋을 나눠줌
print(train, test) #이때 print된 dataframe의 index를 보면 자동으로 shuffle까지 해줌을 알 수 있다고 함.

#torch가 데이터를 읽을 수 있도록 tensor 객체로 변환
X_train = torch.Tensor([[x] for x in list(train.temperature)])
y_train = torch.Tensor([[x] for x in list(train.land_temperature)])
print(X_train, y_train) #확인

#TensorDataset을 이용하여 train_data 변수에 학습데이터를 넣어줌 => DataLoader에 데이터를 넣어줄 준비가 됨.
train_data = TensorDataset(X_train, y_train)
print(train_data)

#result
#<torch.utils.data.dataset.TensorDataset object at 0x7fb469441d10>

#DataLoader로 데이터를 넣어줌.
batch_size = len(X_train) #배치사이즈를 입력해야 하는데 적은 양의 데이터는 배치사이즈를 학습데이터 사이즈로 설정.
train_dl = DataLoader(train_data, batch_size, shuffle=True)

print(train_dl)
#result
#<torch.utils.data.dataloader.DataLoader object at 0x7fb469be9610>

In [ ]:
#선형회귀분석 모델 설정
model = nn.Linear(1, 1) #모델의 input =1, output=1

loss = nn.MSELoss() #비용함수
optimizer = torch.optim.SGD(model.parameters(), lr=0.001) #SGD 경사하강법의 일종

In [ ]:
#설정한 모델에 학습데이터를 넣고 w를 세팅
def lrmodel(epochs, model, loss_fn, optimizer, train_dl):
    for epoch in range(epochs):
        for xb,yb in train_dl:
            pred = model(xb)
            loss = F.mse_loss(pred, yb) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

            optimizer.zero_grad() # cost로 H(x) 개선하는 부분 gradient를 0으로 초기화
            loss.backward()   # backward 연산
            optimizer.step()  # W와 b를 업데이트          
        
        print('Epoch [{}/{}], Cost: {:.4f}'.format(epoch+1, epochs, loss.item()))

epochs=1000
lrmodel(epochs, model, loss, optimizer, train_dl)

In [ ]:
# 학습을 바탕으로 테스트데이터를 사용하여 w와 b의 값을 찾기
print(float(model.weight)) #모델의 w값 추출
print(float(model.bias))   #모델의 b값 추출

sns.regplot(x=df['temperature'], y=df['land_temperature'],fit_reg=True, line_kws={"color":"r"})
#회귀선안그리고 싶으면 fit_reg=False
plt.show()

#reslut
# w=1.1142712831497192
# b=-0.22185981273651123

#result
# w=1.1183185577392578
# b=-0.2816410958766937

#result
#w=1.1170954704284668
#b=-0.25607091188430786

In [ ]:
#최적화가 되었는지 확인하기 위해 x값에 임의의 값 4를 넣어 모델이 예측하는 y의 값을 확인해보는 코드
# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]]) 
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 1.11x 이므로 입력이 4라면 y가 4에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y)

In [ ]:
#참고한 블로그 URL : https://kd1658.tistory.com/113

In [ ]:
#사실 선형회귀는 sckit-learn으로 코드 몇 줄만 돌리면 답을 뽑아낼 수 있다고 한다.
regr = linear_model.LinearRegression()
X = df.temperature.values.reshape(-1,1)
y = df.land_temperature.values.reshape(-1,1)
regr.fit(X, y)
print(regr.coef_[0])
print(regr.intercept_)

# result
# [1.11801535]
# [-0.274278496]